In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

Checking out the dataset that was in two parts and then merging them with the concat() method, since they have the same columns

In [2]:
big_df_pt1 = pd.read_csv('../data/raw/df_final_web_data_pt_1.txt')
big_df_pt2 = pd.read_csv('../data/raw/df_final_web_data_pt_2.txt')
big_df_pt1

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [3]:
#big_df_pt2

In [4]:
merged_df = pd.concat([big_df_pt1, big_df_pt2], axis=0)
merged_df

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [5]:
control = pd.read_csv('../data/clean/client_id_control.csv')
control.head(5)

,client_id
0,1028
1,1104
2,1186
3,1195
4,1197


In [6]:
merged_df = control.merge(merged_df, on = 'client_id', how = 'left')
merged_df

,client_id,visitor_id,visit_id,process_step,date_time
0,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 19:00:26
1,1028,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 19:00:17
2,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 19:00:15
3,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:59:55
4,1028,42237450_62128060588,557292053_87239438319_391157,step_3,2017-04-08 18:58:04
...,...,...,...,...,...
143403,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:27:40
143404,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:26:47
143405,9998346,292425655_16607136645,189177304_69869411700_783154,step_1,2017-03-29 15:26:02
143406,9998346,292425655_16607136645,189177304_69869411700_783154,start,2017-03-29 15:25:00


In [7]:
# Basic inspecting of the dataframe
def inspect_dataframe(merged_df):
    """
    Function to perform basic inspection on a DataFrame: 
    shape, column names, data types, and missing values.
    
    """

    print('Check the shape (rows, columns):')
    print(merged_df.shape)

    print('\nColumn names:')
    print(merged_df.columns)


    print('\nData types:')
    print(merged_df.dtypes)


    print('\nMissing values:')
    print(merged_df.isnull().sum())

# clean column names
def clean_column_names(merged_df):
    """
    Function to clean the column names of a DataFrame:
    - Convert to lowercase
    - Replace spaces with underscores
    - Remove or replace special characters with underscores

    """

    def clean_name(name):
        name = name.lower()
        name = name.replace(" ", "_")
        name = re.sub(r'[^a-z0-9_]', '_', name)
        return name
    
    merged_df.columns = [clean_name(col) for col in merged_df.columns]    
    return merged_df

# check unique and empty values
def check_unique_and_empty(merged_df):
    """
    Function to print the unique and empty values for each column in a DataFrame.
    
    """
    result = []
    
    for column in merged_df.columns:
        unique_values = merged_df[column].nunique()
        empty_values = merged_df[column].isna().sum()
        
        empty_rows = merged_df[column][merged_df[column].isna()].index.tolist()
        
        result.append({
            'Column': column,
            'Unique Values': unique_values,
            'Empty Values': empty_values,
            'Empty Row Indices': empty_rows
        })
    
    merged_df = pd.DataFrame(result)
    
    merged_df.set_index('Column', inplace=True)
    
print(inspect_dataframe(merged_df)) 
print(clean_column_names(merged_df))
print(check_unique_and_empty(merged_df))
display(merged_df)

Check the shape (rows, columns):
(143408, 5)

Column names:
Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

Data types:
client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object

Missing values:
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64
None
        client_id             visitor_id                      visit_id  \
0            1028   42237450_62128060588  557292053_87239438319_391157   
1            1028   42237450_62128060588  557292053_87239438319_391157   
2            1028   42237450_62128060588  557292053_87239438319_391157   
3            1028   42237450_62128060588  557292053_87239438319_391157   
4            1028   42237450_62128060588  557292053_87239438319_391157   
...           ...                    ...                           ...   
143403    9998346  292425655_16607136645  189177304_69869411700

,client_id,visitor_id,visit_id,process_step,date_time
0,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 19:00:26
1,1028,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 19:00:17
2,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 19:00:15
3,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:59:55
4,1028,42237450_62128060588,557292053_87239438319_391157,step_3,2017-04-08 18:58:04
...,...,...,...,...,...
143403,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:27:40
143404,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:26:47
143405,9998346,292425655_16607136645,189177304_69869411700_783154,step_1,2017-03-29 15:26:02
143406,9998346,292425655_16607136645,189177304_69869411700_783154,start,2017-03-29 15:25:00


In [8]:
merged_df.dtypes

client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object

In [9]:
# Changing the date_time column values from objects to 
merged_df['date_time'] = pd.to_datetime(merged_df['date_time'])
merged_df.dtypes

client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object

In [10]:
merged_df = merged_df.sort_values(by=["client_id", "visitor_id", "visit_id", "date_time"])
merged_df

,client_id,visitor_id,visit_id,process_step,date_time
8,1028,42237450_62128060588,557292053_87239438319_391157,start,2017-04-08 18:51:28
7,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:52:17
6,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:53:20
5,1028,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 18:53:29
4,1028,42237450_62128060588,557292053_87239438319_391157,step_3,2017-04-08 18:58:04
...,...,...,...,...,...
143398,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:30:25
143395,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:37:28
143396,9998346,292425655_16607136645,189177304_69869411700_783154,confirm,2017-03-29 15:37:28
143397,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:37:28


In [11]:
# Step 1: Identify 'start' steps that are followed by a 'step_1'
# First, shift the process_step column to check the next step
merged_df['next_step'] = merged_df.groupby('visit_id')['process_step'].shift(-1)

# Step 2: Filter the DataFrame to retain 'start' only if the next step is 'step_1'
filtered_df = merged_df[
    (merged_df['process_step'] != 'start') | (merged_df['next_step'] == 'step_1')
]

# Step 3: Drop the temporary 'next_step' column
filtered_df = filtered_df.drop(columns=['next_step'])

# Step 4: Drop duplicates if needed
filtered_df = filtered_df.drop_duplicates()

# Display the resulting DataFrame
filtered_df

,client_id,visitor_id,visit_id,process_step,date_time
8,1028,42237450_62128060588,557292053_87239438319_391157,start,2017-04-08 18:51:28
7,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:52:17
6,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:53:20
5,1028,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 18:53:29
4,1028,42237450_62128060588,557292053_87239438319_391157,step_3,2017-04-08 18:58:04
...,...,...,...,...,...
143400,9998346,292425655_16607136645,189177304_69869411700_783154,step_1,2017-03-29 15:29:55
143399,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:30:11
143398,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:30:25
143395,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:37:28


### For our project, we need to know how much time a client spends on each step and if they ever reached the last step.

In [12]:
# Valid steps order
valid_steps_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Initialize a list to store the results
time_spent = []

# Function to map steps to their order
def step_order(step):
    return valid_steps_order.index(step)

# Iterate over each unique combination of client, visitor, and visit
for (client_id, visitor_id, visit_id), group in filtered_df.groupby(['client_id', 'visitor_id', 'visit_id']):
    prev_time = None
    prev_step = None
    last_valid_step_order = -1  # Start with an invalid step order to force a check for the first step
    
    for _, row in group.iterrows():
        current_time = row['date_time']
        current_step = row['process_step']
        
        # Initialize is_error to False
        is_error = False
        
        # Check if the current step is a valid step and follows the correct order (no skipping)
        current_step_order = step_order(current_step)
        
        if prev_time is not None:
            # Calculate the time difference between steps
            time_diff = current_time - prev_time
            
            if current_step_order < last_valid_step_order:
                # If the current step goes backwards, just include the row (no error)
                time_spent.append({
                    'client_id': client_id,
                    'visitor_id': visitor_id,
                    'visit_id': visit_id,
                    'from_step': prev_step,
                    'to_step': current_step,
                    'time_spent': time_diff,
                    'is_error': True  # No error for backward steps
                })
            elif current_step_order > last_valid_step_order + 1:
                # If the step order skips (e.g., step_1 → step_3), **skip this row**
                continue  # Skip this row, don't append it
            elif current_step_order == last_valid_step_order:
                # If the steps to and from are the same, **skip this row**
                continue
            else:
                # Valid transition, append to the list
                time_spent.append({
                    'client_id': client_id,
                    'visitor_id': visitor_id,
                    'visit_id': visit_id,
                    'from_step': prev_step,
                    'to_step': current_step,
                    'time_spent': time_diff,
                    'is_error': False  # Valid transition, no error
                })
        
        # Update the last valid step order and previous time/step for the next iteration
        last_valid_step_order = current_step_order
        prev_time = current_time
        prev_step = current_step
        
        if prev_step is None:
            # For the first step, no previous step
            time_spent.append({
                'client_id': client_id,
                'visitor_id': visitor_id,
                'visit_id': visit_id,
                'from_step': None,
                'to_step': current_step,
                'time_spent': None,
                'is_error': False  # No error for the first step
            })

# Create a DataFrame with the results
time_spent_df = pd.DataFrame(time_spent)

# Drop rows with None values in 'from_step' and 'to_step'
time_spent_df = time_spent_df.dropna(subset=['from_step', 'to_step'])

# Display the resulting DataFrame
time_spent_df

,client_id,visitor_id,visit_id,from_step,to_step,time_spent,is_error
0,1028,42237450_62128060588,557292053_87239438319_391157,start,step_1,0 days 00:00:49,False
1,1028,42237450_62128060588,557292053_87239438319_391157,step_1,step_2,0 days 00:01:12,False
2,1028,42237450_62128060588,557292053_87239438319_391157,step_2,step_3,0 days 00:04:35,False
3,1028,42237450_62128060588,557292053_87239438319_391157,step_3,step_1,0 days 00:01:51,True
4,1028,42237450_62128060588,557292053_87239438319_391157,step_1,step_2,0 days 00:00:22,False
...,...,...,...,...,...,...,...
91856,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,step_3,0 days 00:01:13,False
91857,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,step_1,0 days 00:01:55,True
91858,9998346,292425655_16607136645,189177304_69869411700_783154,step_1,step_2,0 days 00:00:16,False
91859,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,step_3,0 days 00:00:14,False


In [13]:
errors_rate = time_spent_df['is_error'].value_counts().values[1] / len(time_spent_df) * 100
print(f'Error rate for the control group: {errors_rate:.2f} %')

Error rate for the control group: 7.90 %


In [14]:
# Filter transitions to include only those that follow the valid steps order
valid_transitions = []

# Iterate over the time_spent_df and filter valid transitions
for _, row in time_spent_df.iterrows():
    from_step = row['from_step']
    to_step = row['to_step']
    
    # Check if the transition follows the valid steps order (i.e., from a step to the next one in order)
    if from_step is not None and to_step is not None:
        if valid_steps_order.index(to_step) == valid_steps_order.index(from_step) + 1:
            valid_transitions.append(row)

# Create a DataFrame with only valid transitions
valid_transitions_df = pd.DataFrame(valid_transitions)

# Calculate the average time spent on each transition
avg_time_spent = valid_transitions_df.groupby(['from_step', 'to_step'])['time_spent'].mean().round(0).reset_index()

# Rename the columns for clarity
avg_time_spent = avg_time_spent.rename(columns={'time_spent': 'avg_time_spent'})

# Display the resulting DataFrame
avg_time_spent

,from_step,to_step,avg_time_spent
0,start,step_1,0 days 00:00:37.643309134
1,step_1,step_2,0 days 00:00:36.139977603
2,step_2,step_3,0 days 00:01:30.402031793
3,step_3,confirm,0 days 00:02:14.906850213


In [15]:
# Filter rows where 'is_error' is True
errors_df = time_spent_df[time_spent_df['is_error'] == True]

# Calculate the average time spent on errors
avg_time_per_error = errors_df['time_spent'].mean()

# Display the results
print(f"Average time spent per error: {avg_time_per_error} seconds")

errors_df

Average time spent per error: 0 days 00:02:54.153305785 seconds


,client_id,visitor_id,visit_id,from_step,to_step,time_spent,is_error
3,1028,42237450_62128060588,557292053_87239438319_391157,step_3,step_1,0 days 00:01:51,True
5,1028,42237450_62128060588,557292053_87239438319_391157,step_2,step_1,0 days 00:00:09,True
15,1197,753759429_54481946928,71862471_21202285428_848395,step_3,step_2,0 days 00:00:11,True
46,7020,131463516_45910398094,61344667_49617029714_100250,step_3,step_2,0 days 00:00:21,True
49,7020,131463516_45910398094,61344667_49617029714_100250,confirm,step_1,0 days 00:00:17,True
...,...,...,...,...,...,...,...
91840,9997391,494669706_3354361161,84654768_90613632047_633963,step_3,step_2,0 days 00:00:17,True
91843,9997470,395791369_55562604618,761490147_96352537762_21814,step_1,start,0 days 00:01:57,True
91845,9997470,395791369_55562604618,761490147_96352537762_21814,step_1,start,0 days 00:02:05,True
91849,9997470,395791369_55562604618,761490147_96352537762_21814,step_3,start,0 days 00:34:15,True


Some clients didn't ever complete the process. Therefore, we need to ensure they are identified and see how many clients reached which step.

In [16]:
# Identify clients who didn't complete the whole process (didn't reach "confirm")
completed_clients = time_spent_df[time_spent_df['to_step'] == 'confirm']['visit_id'].unique()

# Identify clients who didn't reach "confirm"
incomplete_clients = time_spent_df[~time_spent_df['visit_id'].isin(completed_clients)]['visit_id'].unique()

# Show which clients are incomplete
print(f"Clients who didn't complete the process: {incomplete_clients}")

# Check the last step they reached
last_steps = time_spent_df.groupby('visit_id')['to_step'].last()

# Show the last step for each client (whether complete or incomplete)
print(f"Last step for each client: \n{last_steps}")

Clients who didn't complete the process: ['557292053_87239438319_391157' '795373564_99931517312_810896'
 '53540785_76819002881_890663' ... '809603340_74941852314_828669'
 '984781905_40925324211_645419' '84654768_90613632047_633963']
Last step for each client: 
visit_id
100037962_47432393712_705583     step_1
100057941_88477660212_944512     step_3
10006594_66157970412_679648      step_3
10007589_47780784567_391490     confirm
100096068_8301717872_987164      step_1
                                 ...   
999814517_15202944548_822095    confirm
999859408_41720215615_938916    confirm
999890184_77121766521_746360     step_2
999936732_27995195904_324897     step_1
999985675_64610694964_443659    confirm
Name: to_step, Length: 23793, dtype: object


In [17]:
# Get the total number of unique visits
total_visits = time_spent_df['visit_id'].nunique()

# Identify the unique steps, excluding 'start' because it didn't make sense to keep it
steps = ['step_1', 'step_2', 'step_3', 'confirm'] 

# Initialize a dictionary to store the percentage of clients reaching each step
step_percentages = {}

# Iterate through each step and calculate the percentage
for step in steps:
    # Find unique visits that reached the step
    visits_reached_step = time_spent_df[time_spent_df['to_step'] == step]['visit_id'].nunique()
    
    # Calculate the percentage of visits who reached this step
    step_percentages[step] = (visits_reached_step / total_visits) * 100
    print(visits_reached_step)
    print(total_visits)

# Print the results
for step, percentage in step_percentages.items():
    print(f"Percentage of visits who reached {step}: {percentage:.2f}%")

23314
23793
20033
23793
18060
23793
15205
23793
Percentage of visits who reached step_1: 97.99%
Percentage of visits who reached step_2: 84.20%
Percentage of visits who reached step_3: 75.90%
Percentage of visits who reached confirm: 63.91%


## Exporting the tables 

In [18]:
#filtered_df.to_csv('../data/clean/web_data_merged_filtered.csv')

In [19]:
#time_spent_df.to_csv('../data/clean/time_spent.csv')